In [1]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import  Image
import torch
from tqdm import tqdm
import pretrainedmodels
from torch import nn
from torch import optim
from torch.autograd import Variable
from torch.utils.data import Dataset,DataLoader,TensorDataset,ConcatDataset
from torchvision import transforms as trans
from torchvision import models,utils
from pathlib import Path
%matplotlib inline
from dataset.dataset import Dogs

In [2]:
#Parameters
img_size = 331
batch_size = 4
batch_size_top = 4096
use_cuda = torch.cuda.is_available()
model_name = 'nasnet'
learning_rate = 0.0001
dropout_ratio = 0.5
input_shape = 331
crop_mode = 'center'
use_bias = True

In [3]:
#Files Paths
work_folder = Path('../../..')
train_image_folder = work_folder/'train'
test_image_folder = work_folder/'test'
bottlenecks_folder = work_folder/'bottlenecks'
pred_folder = work_folder/'pred'
df_train = pd.read_csv(work_folder/'labels.csv',index_col=0)
df_test = pd.read_csv(work_folder/'sample_submission.csv',index_col=0)


In [4]:
#Select crop mode
if crop_mode == 'center':
    transforms = trans.Compose([
        trans.Resize(input_shape),
        trans.CenterCrop(input_shape),
        trans.ToTensor(),
        trans.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])])
elif crop_mode == 'resize':
    transforms = trans.Compose([
        trans.Resize((input_shape,input_shape)),
        trans.ToTensor(),
        trans.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])])
else:
    transforms = trans.Compose([
        trans.Resize((input_shape,input_shape)),trans.ToTensor()])

In [5]:
#Loading Data
train_dataset = Dogs(train_image_folder,df_train,df_test,is_train=True,resize=False,transforms=transforms)
test_dataset = Dogs(test_image_folder,df_train,df_test,False,resize=False,transforms=transforms)
train_dataset_resize = Dogs(train_image_folder,df_train,df_test,is_train=True,resize=True,transforms=transforms)
train_loader = DataLoader(train_dataset,batch_size,num_workers=0,shuffle=False)
test_loader = DataLoader(test_dataset,batch_size,num_workers=0,shuffle=False)

In [6]:
#Getting NASNet model
def get_extraction_model():
    nasnet = pretrainedmodels.nasnetalarge(num_classes=1000)
    nasnet = nasnet.eval()
    nasnet.avg_pool = nn.AdaptiveAvgPool2d(1)
    del nasnet.dropout
    del nasnet.last_linear
    nasnet.dropout = lambda x:x
    nasnet.last_linear = lambda x:x
    return nasnet

extraction_nasnet = get_extraction_model()

if use_cuda:
    extraction_nasnet.cuda()

In [8]:
#Getting bottleneck features
def get_bottlenecks(data_loader,extration_model,test_mode=False):
    x_pieces = []
    y_pieces = []
    for x,y in tqdm(iter(data_loader)):
        if use_cuda:
            x = Variable(x)
            y = Variable(y) if not test_mode else y
            x = x.cuda()
            y = y.cuda() if not test_mode else y
        x_pieces.append(extration_model(x).cpu().data.numpy())
        y_pieces.append(y.cpu().data.numpy()) if not test_mode else y_pieces
    bottlenecks_x = np.concatenate(x_pieces)
    bottlenecks_y = np.concatenate(y_pieces) if not test_mode else None
    return bottlenecks_x,bottlenecks_y

bottlenecks_x,bottlenecks_y= get_bottlenecks(train_loader,extraction_nasnet)

#Saving bottleneck features
name = "bottleneck"
np.save(bottlenecks_folder/(name+'_x'),bottlenecks_x)
np.save(bottlenecks_folder/(name+'_y'),bottlenecks_y)
bottlenecks_x = np.load(bottlenecks_folder/(name + '_x.npy'))
bottlenecks_y = np.load(bottlenecks_folder/(name + '_y.npy'))

100%|██████████████████████████████████████████████████████████████████████████████| 2556/2556 [05:19<00:00,  7.99it/s]


In [9]:
del extraction_nasnet

In [10]:
##Define Top Layers
class TopModule(nn.Module):
    def __init__(self,dropout_ratio):
        super(TopModule, self).__init__()
        self.aff = nn.Linear(4032, 120,bias=use_bias)
        #self.aff1 = nn.Linear(4032, 1008,bias=use_bias)
        #self.aff2 = nn.Linear(1008, 120,bias=use_bias)
        self.dropout_ratio = dropout_ratio
    def forward(self,x):
        x = nn.Dropout(p = dropout_ratio)(x)
        x = self.aff(x)
        #x = nn.Dropout(p = dropout_ratio)(x)
        #x = self.aff1(x)
        #x = nn.Dropout(p = dropout_ratio)(x)
        #x = self.aff2(x)
        return x

In [11]:
#Splitting into training set and validation set
permutation = np.random.permutation(bottlenecks_x.shape[0])

x_train = bottlenecks_x[permutation][:-int(bottlenecks_x.shape[0]//5)]
x_val = bottlenecks_x[permutation][-int(bottlenecks_x.shape[0]//5):]
y_train = bottlenecks_y[permutation][:-int(bottlenecks_y.shape[0]//5)]
y_val = bottlenecks_y[permutation][-int(bottlenecks_y.shape[0]//5):]

top_only_train_dataset = TensorDataset(torch.FloatTensor(x_train),torch.LongTensor(y_train))
top_only_val_dataset = TensorDataset(torch.FloatTensor(x_val),torch.LongTensor(y_val))

top_only_train_loader = DataLoader(top_only_train_dataset,batch_size=batch_size_top,shuffle=True)
top_only_val_loader = DataLoader(top_only_val_dataset,batch_size=batch_size_top,shuffle=True)

total_dataset = ConcatDataset([top_only_train_dataset,top_only_val_dataset])
total_loader = DataLoader(total_dataset,batch_size=batch_size_top,shuffle=True)

In [12]:
#Getting the top layers and set parameters
top_only_model = TopModule(dropout_ratio)
if use_cuda:
    top_only_model = top_only_model.cuda()
optimizer = optim.Adam(top_only_model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()

In [13]:
#Training the network on the training set
def fit(loader,optimizer,criterion,model=top_only_model,epochs=1500,evaluate=True):
    val_loss_history = []
    val_acc_history = []
    for epoch in range(epochs):  
        running_loss = 0.0
        for i, data in enumerate(loader, 0):
        
            inputs, labels = data
            inputs, labels = Variable(inputs), Variable(labels)
            if use_cuda:
                inputs = inputs.cuda()
                labels = labels.cuda()
        
            optimizer.zero_grad()
        
            # forward + backward 
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()   
        
            optimizer.step()
        
        running_loss += loss.data[0]
        print('[%d, %5d] Train_loss: %.3f' % (epoch+1, i+1, running_loss / len(loader)))
        
        if evaluate:
            model.eval()
            outputs = model(Variable(torch.from_numpy(x_val),volatile=True).cuda() if use_cuda else Variable(torch.from_numpy(x_val),volatile=True))
            labels = torch.from_numpy(y_val).cuda() if use_cuda else torch.from_numpy(y_val)
            labels = Variable(labels,volatile=True)
            loss = criterion(outputs,labels)
            x_val_v = Variable(torch.FloatTensor(x_val),volatile=True).cuda() if use_cuda else Variable(torch.FloatTensor(x_val),volatile=True)
            _,pred = torch.max(model(x_val_v),1)
            val_acc = np.mean(pred.cpu().data.numpy() == labels.cpu().data.numpy())
            val_loss_history.append(loss.cpu().data.numpy())
            val_acc_history.append(val_acc)
    
            print('[%d] Val_loss: %.3f'% (epoch+1, loss))
            print('[%d] Val_acc: %.3f'% (epoch+1, val_acc))
            model.train()
    
    print('Finished Training')
    return val_loss_history,val_acc_history 

val_loss_history,val_acc_history = fit(top_only_train_loader,optimizer,criterion,top_only_model,epochs=1500,evaluate=True)

[1,     2] Train_loss: 2.378
[1] Val_loss: 4.724
[1] Val_acc: 0.033
[2,     2] Train_loss: 2.350
[2] Val_loss: 4.663
[2] Val_acc: 0.071
[3,     2] Train_loss: 2.317
[3] Val_loss: 4.606
[3] Val_acc: 0.123
[4,     2] Train_loss: 2.286
[4] Val_loss: 4.542
[4] Val_acc: 0.204
[5,     2] Train_loss: 2.253
[5] Val_loss: 4.477
[5] Val_acc: 0.287
[6,     2] Train_loss: 2.224
[6] Val_loss: 4.420
[6] Val_acc: 0.407
[7,     2] Train_loss: 2.192
[7] Val_loss: 4.354
[7] Val_acc: 0.516
[8,     2] Train_loss: 2.161
[8] Val_loss: 4.297
[8] Val_acc: 0.618
[9,     2] Train_loss: 2.131
[9] Val_loss: 4.241
[9] Val_acc: 0.700
[10,     2] Train_loss: 2.103
[10] Val_loss: 4.177
[10] Val_acc: 0.775
[11,     2] Train_loss: 2.073
[11] Val_loss: 4.117
[11] Val_acc: 0.813
[12,     2] Train_loss: 2.041
[12] Val_loss: 4.051
[12] Val_acc: 0.841
[13,     2] Train_loss: 2.010
[13] Val_loss: 3.995
[13] Val_acc: 0.871
[14,     2] Train_loss: 1.982
[14] Val_loss: 3.932
[14] Val_acc: 0.886
[15,     2] Train_loss: 1.951
[15

[116,     2] Train_loss: 0.255
[116] Val_loss: 0.514
[116] Val_acc: 0.957
[117,     2] Train_loss: 0.247
[117] Val_loss: 0.503
[117] Val_acc: 0.958
[118,     2] Train_loss: 0.241
[118] Val_loss: 0.493
[118] Val_acc: 0.958
[119,     2] Train_loss: 0.235
[119] Val_loss: 0.495
[119] Val_acc: 0.959
[120,     2] Train_loss: 0.241
[120] Val_loss: 0.484
[120] Val_acc: 0.956
[121,     2] Train_loss: 0.231
[121] Val_loss: 0.476
[121] Val_acc: 0.955
[122,     2] Train_loss: 0.230
[122] Val_loss: 0.473
[122] Val_acc: 0.958
[123,     2] Train_loss: 0.227
[123] Val_loss: 0.462
[123] Val_acc: 0.957
[124,     2] Train_loss: 0.227
[124] Val_loss: 0.460
[124] Val_acc: 0.955
[125,     2] Train_loss: 0.223
[125] Val_loss: 0.458
[125] Val_acc: 0.956
[126,     2] Train_loss: 0.220
[126] Val_loss: 0.449
[126] Val_acc: 0.956
[127,     2] Train_loss: 0.216
[127] Val_loss: 0.448
[127] Val_acc: 0.957
[128,     2] Train_loss: 0.213
[128] Val_loss: 0.439
[128] Val_acc: 0.956
[129,     2] Train_loss: 0.211
[129] V

[226] Val_acc: 0.957
[227,     2] Train_loss: 0.107
[227] Val_loss: 0.234
[227] Val_acc: 0.958
[228,     2] Train_loss: 0.107
[228] Val_loss: 0.234
[228] Val_acc: 0.958
[229,     2] Train_loss: 0.103
[229] Val_loss: 0.229
[229] Val_acc: 0.957
[230,     2] Train_loss: 0.101
[230] Val_loss: 0.233
[230] Val_acc: 0.956
[231,     2] Train_loss: 0.098
[231] Val_loss: 0.231
[231] Val_acc: 0.958
[232,     2] Train_loss: 0.101
[232] Val_loss: 0.231
[232] Val_acc: 0.958
[233,     2] Train_loss: 0.100
[233] Val_loss: 0.231
[233] Val_acc: 0.958
[234,     2] Train_loss: 0.101
[234] Val_loss: 0.226
[234] Val_acc: 0.957
[235,     2] Train_loss: 0.099
[235] Val_loss: 0.227
[235] Val_acc: 0.954
[236,     2] Train_loss: 0.106
[236] Val_loss: 0.228
[236] Val_acc: 0.957
[237,     2] Train_loss: 0.100
[237] Val_loss: 0.227
[237] Val_acc: 0.958
[238,     2] Train_loss: 0.099
[238] Val_loss: 0.228
[238] Val_acc: 0.960
[239,     2] Train_loss: 0.098
[239] Val_loss: 0.225
[239] Val_acc: 0.958
[240,     2] Trai

[337] Val_loss: 0.185
[337] Val_acc: 0.957
[338,     2] Train_loss: 0.074
[338] Val_loss: 0.188
[338] Val_acc: 0.959
[339,     2] Train_loss: 0.074
[339] Val_loss: 0.185
[339] Val_acc: 0.957
[340,     2] Train_loss: 0.076
[340] Val_loss: 0.187
[340] Val_acc: 0.961
[341,     2] Train_loss: 0.069
[341] Val_loss: 0.186
[341] Val_acc: 0.961
[342,     2] Train_loss: 0.073
[342] Val_loss: 0.186
[342] Val_acc: 0.956
[343,     2] Train_loss: 0.073
[343] Val_loss: 0.187
[343] Val_acc: 0.957
[344,     2] Train_loss: 0.074
[344] Val_loss: 0.187
[344] Val_acc: 0.959
[345,     2] Train_loss: 0.073
[345] Val_loss: 0.182
[345] Val_acc: 0.956
[346,     2] Train_loss: 0.070
[346] Val_loss: 0.187
[346] Val_acc: 0.959
[347,     2] Train_loss: 0.074
[347] Val_loss: 0.188
[347] Val_acc: 0.959
[348,     2] Train_loss: 0.071
[348] Val_loss: 0.184
[348] Val_acc: 0.958
[349,     2] Train_loss: 0.071
[349] Val_loss: 0.186
[349] Val_acc: 0.957
[350,     2] Train_loss: 0.073
[350] Val_loss: 0.181
[350] Val_acc: 0

[448,     2] Train_loss: 0.061
[448] Val_loss: 0.171
[448] Val_acc: 0.956
[449,     2] Train_loss: 0.054
[449] Val_loss: 0.172
[449] Val_acc: 0.956
[450,     2] Train_loss: 0.055
[450] Val_loss: 0.171
[450] Val_acc: 0.957
[451,     2] Train_loss: 0.059
[451] Val_loss: 0.171
[451] Val_acc: 0.957
[452,     2] Train_loss: 0.057
[452] Val_loss: 0.170
[452] Val_acc: 0.956
[453,     2] Train_loss: 0.056
[453] Val_loss: 0.172
[453] Val_acc: 0.957
[454,     2] Train_loss: 0.056
[454] Val_loss: 0.169
[454] Val_acc: 0.959
[455,     2] Train_loss: 0.060
[455] Val_loss: 0.169
[455] Val_acc: 0.959
[456,     2] Train_loss: 0.057
[456] Val_loss: 0.173
[456] Val_acc: 0.955
[457,     2] Train_loss: 0.058
[457] Val_loss: 0.172
[457] Val_acc: 0.960
[458,     2] Train_loss: 0.055
[458] Val_loss: 0.169
[458] Val_acc: 0.956
[459,     2] Train_loss: 0.059
[459] Val_loss: 0.172
[459] Val_acc: 0.958
[460,     2] Train_loss: 0.059
[460] Val_loss: 0.167
[460] Val_acc: 0.958
[461,     2] Train_loss: 0.058
[461] V

...

[1001,     2] Train_loss: 0.029
[1001] Val_loss: 0.160
[1001] Val_acc: 0.955
[1002,     2] Train_loss: 0.027
[1002] Val_loss: 0.162
[1002] Val_acc: 0.958
[1003,     2] Train_loss: 0.027
[1003] Val_loss: 0.165
[1003] Val_acc: 0.955
[1004,     2] Train_loss: 0.028
[1004] Val_loss: 0.161
[1004] Val_acc: 0.957
[1005,     2] Train_loss: 0.026
[1005] Val_loss: 0.156
[1005] Val_acc: 0.955
[1006,     2] Train_loss: 0.028
[1006] Val_loss: 0.160
[1006] Val_acc: 0.957
[1007,     2] Train_loss: 0.027
[1007] Val_loss: 0.162
[1007] Val_acc: 0.954
[1008,     2] Train_loss: 0.030
[1008] Val_loss: 0.163
[1008] Val_acc: 0.955
[1009,     2] Train_loss: 0.029
[1009] Val_loss: 0.164
[1009] Val_acc: 0.956
[1010,     2] Train_loss: 0.029
[1010] Val_loss: 0.163
[1010] Val_acc: 0.955
[1011,     2] Train_loss: 0.027
[1011] Val_loss: 0.162
[1011] Val_acc: 0.958
[1012,     2] Train_loss: 0.028
[1012] Val_loss: 0.164
[1012] Val_acc: 0.958
[1013,     2] Train_loss: 0.028
[1013] Val_loss: 0.165
[1013] Val_acc: 0.958

[1107,     2] Train_loss: 0.028
[1107] Val_loss: 0.165
[1107] Val_acc: 0.955
[1108,     2] Train_loss: 0.024
[1108] Val_loss: 0.167
[1108] Val_acc: 0.958
[1109,     2] Train_loss: 0.026
[1109] Val_loss: 0.164
[1109] Val_acc: 0.956
[1110,     2] Train_loss: 0.026
[1110] Val_loss: 0.163
[1110] Val_acc: 0.958
[1111,     2] Train_loss: 0.029
[1111] Val_loss: 0.164
[1111] Val_acc: 0.958
[1112,     2] Train_loss: 0.027
[1112] Val_loss: 0.166
[1112] Val_acc: 0.957
[1113,     2] Train_loss: 0.026
[1113] Val_loss: 0.165
[1113] Val_acc: 0.955
[1114,     2] Train_loss: 0.025
[1114] Val_loss: 0.168
[1114] Val_acc: 0.956
[1115,     2] Train_loss: 0.028
[1115] Val_loss: 0.164
[1115] Val_acc: 0.956
[1116,     2] Train_loss: 0.026
[1116] Val_loss: 0.163
[1116] Val_acc: 0.957
[1117,     2] Train_loss: 0.027
[1117] Val_loss: 0.162
[1117] Val_acc: 0.956
[1118,     2] Train_loss: 0.026
[1118] Val_loss: 0.165
[1118] Val_acc: 0.955
[1119,     2] Train_loss: 0.024
[1119] Val_loss: 0.162
[1119] Val_acc: 0.957

[1213,     2] Train_loss: 0.025
[1213] Val_loss: 0.165
[1213] Val_acc: 0.957
[1214,     2] Train_loss: 0.024
[1214] Val_loss: 0.168
[1214] Val_acc: 0.955
[1215,     2] Train_loss: 0.022
[1215] Val_loss: 0.166
[1215] Val_acc: 0.956
[1216,     2] Train_loss: 0.022
[1216] Val_loss: 0.164
[1216] Val_acc: 0.955
[1217,     2] Train_loss: 0.024
[1217] Val_loss: 0.170
[1217] Val_acc: 0.958
[1218,     2] Train_loss: 0.024
[1218] Val_loss: 0.167
[1218] Val_acc: 0.957
[1219,     2] Train_loss: 0.023
[1219] Val_loss: 0.161
[1219] Val_acc: 0.955
[1220,     2] Train_loss: 0.024
[1220] Val_loss: 0.164
[1220] Val_acc: 0.956
[1221,     2] Train_loss: 0.024
[1221] Val_loss: 0.169
[1221] Val_acc: 0.956
[1222,     2] Train_loss: 0.022
[1222] Val_loss: 0.162
[1222] Val_acc: 0.956
[1223,     2] Train_loss: 0.022
[1223] Val_loss: 0.167
[1223] Val_acc: 0.954
[1224,     2] Train_loss: 0.024
[1224] Val_loss: 0.169
[1224] Val_acc: 0.956
[1225,     2] Train_loss: 0.024
[1225] Val_loss: 0.166
[1225] Val_acc: 0.958

[1319,     2] Train_loss: 0.022
[1319] Val_loss: 0.163
[1319] Val_acc: 0.955
[1320,     2] Train_loss: 0.022
[1320] Val_loss: 0.167
[1320] Val_acc: 0.956
[1321,     2] Train_loss: 0.021
[1321] Val_loss: 0.163
[1321] Val_acc: 0.955
[1322,     2] Train_loss: 0.022
[1322] Val_loss: 0.169
[1322] Val_acc: 0.955
[1323,     2] Train_loss: 0.021
[1323] Val_loss: 0.165
[1323] Val_acc: 0.955
[1324,     2] Train_loss: 0.020
[1324] Val_loss: 0.169
[1324] Val_acc: 0.957
[1325,     2] Train_loss: 0.022
[1325] Val_loss: 0.167
[1325] Val_acc: 0.955
[1326,     2] Train_loss: 0.021
[1326] Val_loss: 0.165
[1326] Val_acc: 0.956
[1327,     2] Train_loss: 0.020
[1327] Val_loss: 0.165
[1327] Val_acc: 0.957
[1328,     2] Train_loss: 0.020
[1328] Val_loss: 0.168
[1328] Val_acc: 0.953
[1329,     2] Train_loss: 0.021
[1329] Val_loss: 0.166
[1329] Val_acc: 0.956
[1330,     2] Train_loss: 0.022
[1330] Val_loss: 0.166
[1330] Val_acc: 0.958
[1331,     2] Train_loss: 0.022
[1331] Val_loss: 0.167
[1331] Val_acc: 0.957

[1425,     2] Train_loss: 0.020
[1425] Val_loss: 0.170
[1425] Val_acc: 0.958
[1426,     2] Train_loss: 0.018
[1426] Val_loss: 0.161
[1426] Val_acc: 0.957
[1427,     2] Train_loss: 0.018
[1427] Val_loss: 0.171
[1427] Val_acc: 0.957
[1428,     2] Train_loss: 0.018
[1428] Val_loss: 0.168
[1428] Val_acc: 0.955
[1429,     2] Train_loss: 0.018
[1429] Val_loss: 0.170
[1429] Val_acc: 0.955
[1430,     2] Train_loss: 0.020
[1430] Val_loss: 0.167
[1430] Val_acc: 0.957
[1431,     2] Train_loss: 0.019
[1431] Val_loss: 0.168
[1431] Val_acc: 0.957
[1432,     2] Train_loss: 0.019
[1432] Val_loss: 0.167
[1432] Val_acc: 0.954
[1433,     2] Train_loss: 0.019
[1433] Val_loss: 0.165
[1433] Val_acc: 0.957
[1434,     2] Train_loss: 0.020
[1434] Val_loss: 0.165
[1434] Val_acc: 0.956
[1435,     2] Train_loss: 0.018
[1435] Val_loss: 0.171
[1435] Val_acc: 0.956
[1436,     2] Train_loss: 0.019
[1436] Val_loss: 0.168
[1436] Val_acc: 0.958
[1437,     2] Train_loss: 0.019
[1437] Val_loss: 0.164
[1437] Val_acc: 0.955

In [14]:
#Get the best epochs and train again on the whole training data(train + validation)
best_epochs = np.argmin(np.array(val_loss_history))
fit(total_loader,optimizer,criterion,top_only_model,epochs=best_epochs,evaluate=False)

[1,     3] Train_loss: 0.018
[2,     3] Train_loss: 0.021
[3,     3] Train_loss: 0.023
[4,     3] Train_loss: 0.022
[5,     3] Train_loss: 0.023
[6,     3] Train_loss: 0.026
[7,     3] Train_loss: 0.018
[8,     3] Train_loss: 0.022
[9,     3] Train_loss: 0.020
[10,     3] Train_loss: 0.019
[11,     3] Train_loss: 0.021
[12,     3] Train_loss: 0.018
[13,     3] Train_loss: 0.019
[14,     3] Train_loss: 0.017
[15,     3] Train_loss: 0.017
[16,     3] Train_loss: 0.018
[17,     3] Train_loss: 0.018
[18,     3] Train_loss: 0.013
[19,     3] Train_loss: 0.016
[20,     3] Train_loss: 0.019
[21,     3] Train_loss: 0.019
[22,     3] Train_loss: 0.017
[23,     3] Train_loss: 0.021
[24,     3] Train_loss: 0.020
[25,     3] Train_loss: 0.020
[26,     3] Train_loss: 0.018
[27,     3] Train_loss: 0.018
[28,     3] Train_loss: 0.018
[29,     3] Train_loss: 0.016
[30,     3] Train_loss: 0.016
[31,     3] Train_loss: 0.018
[32,     3] Train_loss: 0.018
[33,     3] Train_loss: 0.023
[34,     3] Train_l

[268,     3] Train_loss: 0.010
[269,     3] Train_loss: 0.015
[270,     3] Train_loss: 0.012
[271,     3] Train_loss: 0.013
[272,     3] Train_loss: 0.013
[273,     3] Train_loss: 0.011
[274,     3] Train_loss: 0.011
[275,     3] Train_loss: 0.012
[276,     3] Train_loss: 0.013
[277,     3] Train_loss: 0.014
[278,     3] Train_loss: 0.012
[279,     3] Train_loss: 0.014
[280,     3] Train_loss: 0.012
[281,     3] Train_loss: 0.012
[282,     3] Train_loss: 0.013
[283,     3] Train_loss: 0.012
[284,     3] Train_loss: 0.012
[285,     3] Train_loss: 0.013
[286,     3] Train_loss: 0.012
[287,     3] Train_loss: 0.012
[288,     3] Train_loss: 0.010
[289,     3] Train_loss: 0.012
[290,     3] Train_loss: 0.013
[291,     3] Train_loss: 0.012
[292,     3] Train_loss: 0.011
[293,     3] Train_loss: 0.010
[294,     3] Train_loss: 0.011
[295,     3] Train_loss: 0.012
[296,     3] Train_loss: 0.011
[297,     3] Train_loss: 0.011
[298,     3] Train_loss: 0.010
[299,     3] Train_loss: 0.012
[300,   

[532,     3] Train_loss: 0.008
[533,     3] Train_loss: 0.009
[534,     3] Train_loss: 0.009
[535,     3] Train_loss: 0.008
[536,     3] Train_loss: 0.011
[537,     3] Train_loss: 0.009
[538,     3] Train_loss: 0.007
[539,     3] Train_loss: 0.008
[540,     3] Train_loss: 0.008
[541,     3] Train_loss: 0.009
[542,     3] Train_loss: 0.010
[543,     3] Train_loss: 0.009
[544,     3] Train_loss: 0.011
[545,     3] Train_loss: 0.008
[546,     3] Train_loss: 0.009
[547,     3] Train_loss: 0.008
[548,     3] Train_loss: 0.008
[549,     3] Train_loss: 0.009
[550,     3] Train_loss: 0.010
[551,     3] Train_loss: 0.010
[552,     3] Train_loss: 0.008
[553,     3] Train_loss: 0.009
[554,     3] Train_loss: 0.008
[555,     3] Train_loss: 0.009
[556,     3] Train_loss: 0.008
[557,     3] Train_loss: 0.007
[558,     3] Train_loss: 0.009
[559,     3] Train_loss: 0.010
[560,     3] Train_loss: 0.009
[561,     3] Train_loss: 0.009
[562,     3] Train_loss: 0.008
[563,     3] Train_loss: 0.008
[564,   

([], [])

In [15]:
#Get NASNet model
extraction_nasnet = get_extraction_model()
if use_cuda:
    extraction_nasnet.cuda()
    
#Get Test Bottleneck features and save them
bottlenecks_test_x,test_y = get_bottlenecks(test_loader,extraction_nasnet,True)
np.save(bottlenecks_folder/(name+'_test_x'),bottlenecks_test_x)

100%|██████████████████████████████████████████████████████████████████████████████| 2590/2590 [05:23<00:00,  8.01it/s]


In [16]:
del extraction_nasnet

In [17]:
#Generating Result csv file
top_only_model.eval()
x_test = Variable(torch.FloatTensor(bottlenecks_test_x),volatile=True).cuda() if use_cuda else Variable(torch.FloatTensor(bottlenecks_test_x),volatile=True)
pred_np = (nn.Softmax(1)(top_only_model(x_test))).cpu().data.numpy()
df_pred = pd.DataFrame(pred_np,index=df_test.index,columns=df_test.columns)
df_pred.to_csv("pred"+'.csv')